In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup # commonly used for parsing HTML content

import re
import spacy
from spacy.tokens import DocBin

In [2]:
!python -m spacy download en_core_web_lg # install the missing model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# Get resume links from link of pages
def get_resume(url_web='https://www.jobspider.com/job/resume-search-results.asp/words_Software%2BEngineer'):
    href_list = []
    print(f'The function takes 30 pages, which is the maximum at the present time of {url_web}')

    for j in range(1, 30):
        url_page = url_web + f'/page_{j}'

        response = requests.get(url_page)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all links that we need on the page
        href_locals = soup.find_all('a', attrs={'href': True}, href=lambda value: value.startswith('/job/view-resume-'))
        try:
            for href_local in href_locals:
                href_list.append('https://www.jobspider.com' + href_local['href'])
        except:
            pass

    return href_list

# Convert resume into text
def resume_to_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'lxml')

    # Get job function sought
    resume_title = soup.find_all(["h1"])[0].text.strip()

    # Get job decription
    decriptions = soup.find_all(["td"])
    return url, resume_title, [decription.text.strip() for decription in decriptions]

In [4]:
def preprocess(decription):
    text = "\n".join(decription[19:-3])                                   # omit data after viz
    text = re.sub(r'\n\s*\n', '\n', text)                                 # Replace multiple newlines with one
    text = re.sub(r'\s{2,}', ' ', text)                                   # Replace multiple spaces with one
    text = text.replace('\t', ' ')                                        # Replace tabs with one space
    text = "".join([s for s in text.splitlines(True) if s.strip("\r\n")]) # filters out any empty lines
    text = re.sub('[^A-Za-z0-9\n]+', ' ', text)                           # replace any characters that are not alphanumeric

    return text

In [5]:
href_list = get_resume() # default is https://www.jobspider.com/job/resume-search-results.asp/words_Software%2BEngineer

# List to store information
urls = []
resume_types = []
decriptions = []

data = []
nlp_temp = spacy.load("en_core_web_lg")

for href in tqdm(href_list):
    url, resume_title, description = resume_to_text(href)

    _description = preprocess(description) # preprocessing base on obseravtion
    doc = nlp_temp(_description) # fot getting ner in _description
    row = [url, resume_title, _description, {'entities': [(ent.start_char, ent.end_char, ent.text, ent.label_, str(spacy.explain(ent.label_))) for ent in doc.ents]}]

    data.append(row)

The function takes 30 pages, which is the maximum at the present time of https://www.jobspider.com/job/resume-search-results.asp/words_Software%2BEngineer


100%|██████████| 1450/1450 [09:27<00:00,  2.55it/s]


In [6]:
import pickle
pickle.dump(data, open('scrapping_data_lg_update.json','wb'))

In [7]:
data[0]

['https://www.jobspider.com/job/view-resume-84752.html',
 'Business Administration Resume',
 'Business Administration Resume\nDesired Industry Human Resources\nSpiderID 84752\nDesired Job Location Huntersville North Carolina\nDate Posted 10 4 2024\nType of Position Full Time Permanent\nAvailability Date Immediately\nDesired Wage 60000\nU S Work Authorization Yes\nJob Level Experienced with over 2 years experience\nWilling to Travel Yes Less Than 25 \nHighest Degree Attained Other\nWilling to Relocate Yes\nObjective Analytical detail oriented and results driven professional with hands on experience in managing business operations and leading teams within the retail industry Armed with an in depth understanding of financial management budgeting and accounting principles as well as regulatory compliance and ethical business practices Adaptable individual capable of adjusting to changing market conditions and embracing innovations and strategic initiatives for business growth Equipped with

In [8]:
import pickle

with open("scrapping_data_lg_update.json", "rb") as f:
    data = pickle.load(f)

In [9]:
def get_spacy_doc(file, data):

  nlp = spacy.blank("en")
  db = DocBin()

  for url, resume_tittle, text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, tex, l, explain in annot:
      label = explain
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label = label, alignment_mode = 'strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + '\n'
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [10]:
from sklearn.model_selection import train_test_split
train, dev = train_test_split(data, test_size = 0.2, random_state = 42)

In [11]:
file = open('train_file.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train.spacy')

db = get_spacy_doc(file, dev)
db.to_disk('dev.spacy')

file.close()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
100%|██████████| 290/290 [00:16<00:00, 17.68it/s]


In [12]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [13]:
# Initialization setting
!python -m spacy init fill-config base_config.cfg config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [14]:
# This took approximately 8 hour for training on google colab pro gpu mode
!python -m spacy train config.cfg  --output generated_model --paths.train train.spacy --paths.dev dev.spacy --gpu-id 0

ℹ Saving to output directory: generated_model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 122kB/s]
config.json: 

In [16]:
print('hello')

hello
hello


In [ ]:
model = spacy.load('generated_model/model-best')

ValueError: [E002] Can't find factory for 'transformer' for language English (en). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, entity_ruler, tagger, morphologizer, ner, beam_ner, senter, sentencizer, spancat, spancat_singlelabel, span_finder, future_entity_ruler, span_ruler, textcat, textcat_multilabel, en.lemmatizer

In [ ]:
# Test example
main_str = ['''Manufacturing Production Manager Resume
Desired Industry Manufacturing
SpiderID 78692
Desired Job Location Windsor Colorado
Date Posted 1 23 2017
Type of Position Full Time Permanent
Availability Date Immediately
Desired Wage 95000
U S Work Authorization Yes
Job Level Management Manager Director
Willing to Travel
Highest Degree Attained Other
Willing to Relocate Yes
Objective Dynamic and growth driven professional offering hands on management experience and comprehensive background in manufacturing and engineering operations within highly competitive setting Adept at reengineering unproductive work processes as well as in planning and implementing various sustainable and cost effective work programs to drive continuous improvement of operations Armed with exceptional organizational and critical problem solving aptitudes to formulate effective solutions on complex production and quality issues Equipped with tactical leadership capabilities in supervising and guiding teams toward the successful and timely completion of projects Proficient with Microsoft Office applications Project Management and Microsoft Visio
Experience Relevant ExperienceVestas Blades America Windsor COProduction Engineer Jan 2014 PresentContribute efforts in achieving production plan budget tooling equipment and bill of material along with quality control and safety Conceptualized new methods to optimize production levels while maintaining production costs yields quality and safety Identified and resolved process problems with effective solutions which decreased downtime and minimized costs Initiated plant trials to measure performance capabilities while ensuring updated documentation of process procedures Leveraged industry expertise in streamlining the manufacturing of turbine blades Production Supervisor Mar 2009 Jan 2014Rendered oversight to more than 70 employees to guarantee accordance of operation with production plan and goals Observed strict compliance with safety and quality guidelines and handled inventory control created schedules delegated work and facilitated training of staff Generated production reports for production and operations managers regarding production areas performance Served as a Shells Lighthouse project member while drafting and modifying all standard operating procedures Functioned as department lead for production quality training and process improvements Closely monitored operations productivity to determine areas for improvement in overall production process Pioneered the development of all shells production process job cards that decreased process times and improved efficiency and quality through changed production flow layout Anheuser Busch Fort Collis COBrew House Control Panel Operator Mar 2007 Mar 2009Efficiently administered beer brewing process from raw material selection and recipe formulation through the use of Siemens software Strictly enforced standard operating procedures and safe working practices Determined and evaluated all critical control points to achieve consistent product quality within allotted schedules Other ExperienceUnited States Airways Denver CoCustomer Service Representative Kroger Co King Soopers Smiths Food Drug Fort Collins CoGrocery Manager Head Clerk Night Crew Manager Front End ManagerInventory Control Manager Warehouse Manager
Education EducationAssociate of Science with emphasis in chemistry and biologyFront Range Community College Fort Collins COPharmacy Pre Requisites for Doctor of Pharmacy Pharm D University of Wyoming Laramie WYPharmacy Pre Requisites for Doctor of Pharmacy Pharm D Western Wyoming Community College Rock Springs WY Deans Honor RollProfessional DevelopmentCertificationsSupervisor Certification Six Sigma Yellow Belt Project ManagementQuality Management Crucial Conversations Fort Lift License Crane License
Affiliations
Skills Manufacturing Production Manager Project Planning Cost Reduction and Budget Optimization Resource Allocation Six Sigma Quality ControlLean Manufacturing Plant Management Manufacturing Inspection Administration Cross functional Team Building
Additional Information TrainingProduction Instructor Coordinator Planner Coordinator Wrote Training Document
Reference Available upon request
Candidate Contact Information
JobSpider com has chosen not to make contact information available on this page Click Contact Candidate to send this candidate a response
Manufacturing Production Manager Resume
Desired Industry Manufacturing
SpiderID 78692
Desired Job Location Windsor Colorado
Date Posted 1 23 2017
Type of Position Full Time Permanent
Availability Date Immediately
Desired Wage 95000
U S Work Authorization Yes
Job Level Management Manager Director
Willing to Travel
Highest Degree Attained Other
Willing to Relocate Yes
Objective Dynamic and growth driven professional offering hands on management experience and comprehensive background in manufacturing and engineering operations within highly competitive setting Adept at reengineering unproductive work processes as well as in planning and implementing various sustainable and cost effective work programs to drive continuous improvement of operations Armed with exceptional organizational and critical problem solving aptitudes to formulate effective solutions on complex production and quality issues Equipped with tactical leadership capabilities in supervising and guiding teams toward the successful and timely completion of projects Proficient with Microsoft Office applications Project Management and Microsoft Visio
Experience Relevant ExperienceVestas Blades America Windsor COProduction Engineer Jan 2014 PresentContribute efforts in achieving production plan budget tooling equipment and bill of material along with quality control and safety Conceptualized new methods to optimize production levels while maintaining production costs yields quality and safety Identified and resolved process problems with effective solutions which decreased downtime and minimized costs Initiated plant trials to measure performance capabilities while ensuring updated documentation of process procedures Leveraged industry expertise in streamlining the manufacturing of turbine blades Production Supervisor Mar 2009 Jan 2014Rendered oversight to more than 70 employees to guarantee accordance of operation with production plan and goals Observed strict compliance with safety and quality guidelines and handled inventory control created schedules delegated work and facilitated training of staff Generated production reports for production and operations managers regarding production areas performance Served as a Shells Lighthouse project member while drafting and modifying all standard operating procedures Functioned as department lead for production quality training and process improvements Closely monitored operations productivity to determine areas for improvement in overall production process Pioneered the development of all shells production process job cards that decreased process times and improved efficiency and quality through changed production flow layout Anheuser Busch Fort Collis COBrew House Control Panel Operator Mar 2007 Mar 2009Efficiently administered beer brewing process from raw material selection and recipe formulation through the use of Siemens software Strictly enforced standard operating procedures and safe working practices Determined and evaluated all critical control points to achieve consistent product quality within allotted schedules Other ExperienceUnited States Airways Denver CoCustomer Service Representative Kroger Co King Soopers Smiths Food Drug Fort Collins CoGrocery Manager Head Clerk Night Crew Manager Front End ManagerInventory Control Manager Warehouse Manager
Education EducationAssociate of Science with emphasis in chemistry and biologyFront Range Community College Fort Collins COPharmacy Pre Requisites for Doctor of Pharmacy Pharm D University of Wyoming Laramie WYPharmacy Pre Requisites for Doctor of Pharmacy Pharm D Western Wyoming Community College Rock Springs WY Deans Honor RollProfessional DevelopmentCertificationsSupervisor Certification Six Sigma Yellow Belt Project ManagementQuality Management Crucial Conversations Fort Lift License Crane License
Affiliations
Skills Manufacturing Production Manager Project Planning Cost Reduction and Budget Optimization Resource Allocation Six Sigma Quality ControlLean Manufacturing Plant Management Manufacturing Inspection Administration Cross functional Team Building
Additional Information TrainingProduction Instructor Coordinator Planner Coordinator Wrote Training Document
Reference Available upon request
Candidate Contact Information
JobSpider com has chosen not to make contact information available on this page Click Contact Candidate to send this candidate a response''' ]


In [ ]:
for doc in nlp.pipe(main_str, disable=["tagger", "parser"]):
  for ent in doc.ents:
    print((ent.text,ent.label_))
    # print([(ent.text, ent.label_) for ent in doc.ents])


('Manufacturing Production Manager', 'JOB_TITLE')
('Manager Director', 'JOB_TITLE')
('manufacturing and engineering operations', 'SKILL')
('reengineering unproductive work processes as well as in planning and implementing various sustainable and cost effective work programs to drive continuous improvement of operations', 'SKILL')
('Microsoft Office applications Project Management', 'TOOL')
('Microsoft Visio', 'TOOL')
('ExperienceVestas Blades America', 'ORG')
('COProduction Engineer Jan 2014 PresentContribute efforts in achieving production plan budget tooling equipment and bill of material along with quality control and safety', 'EXPERIENCE')
('EducationAssociate of Science with emphasis in chemistry', 'DEGREE')
('biologyFront Range Community College Fort Collins', 'ORG')
('Doctor of Pharmacy Pharm D', 'DEGREE')
('Doctor of Pharmacy Pharm D', 'DEGREE')
('Manufacturing Production ManagerProject', 'SKILL')
('Manufacturing Production Manager', 'JOB_TITLE')
('Manager Director', 'JOB_TITLE